# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [6]:
#Answers for PA1-Q5

a) how many faculty taught COSI courses last year?

In [7]:
len({tuple(course['instructor']) for course in courses if course['subject'] == 'COSI'})

27

b)what is the total number of students taking COSI courses last year? 

In [8]:
student_cs = [course['enrolled'] for course in courses if course['subject'] == 'COSI']
sum([i for i in student_cs])

2223

c) what was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [9]:
import statistics as st
st.median([int(course['enrolled']) for course in courses if course['subject'] == 'COSI' and course['enrolled']>10])

38.0

d) create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [10]:
from itertools import groupby
subject_and_enrolled = [(k, sum([i["enrolled"] for i in list(v)])) for k,v in groupby(courses,key=lambda courses:courses['subject'])]
subject_and_enrolled.sort(key = lambda subject_and_enrolled: -subject_and_enrolled[1])
print(subject_and_enrolled[:10])

[('HWL', 1167), ('HS', 1005), ('BIOL', 956), ('CHEM', 898), ('HS', 867), ('CHEM', 841), ('HWL', 784), ('ECON', 671), ('HWL', 653), ('HS', 592)]


e)do the same as in (d) but print the top 10 subjects in terms of number of courses offered

In [11]:
from collections import Counter
course_by_subject = sorted(courses, key = lambda course: course['subject'])
subject = [c['subject'] for c in course_by_subject]
Counter(subject).most_common()[:10]

[('BIOL', 613),
 ('HIST', 498),
 ('PSYC', 417),
 ('NEUR', 403),
 ('BCHM', 296),
 ('PHYS', 288),
 ('HS', 274),
 ('COSI', 272),
 ('MUS', 266),
 ('ENG', 265)]

f)do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject

In [12]:
instructor = list({(c['subject'],c['instructor']) for c in courses})
instructor.sort()
lst = list()
for key, val in instructor :
    lst.append(key)
Counter(lst).most_common()[:10]

[('HS', 87),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('BUS', 47),
 ('HIST', 47),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

g)list the top 20 faculty in terms of number of students they taught

In [13]:
course_by_size = sorted(courses, key = lambda course: -course['enrolled'])
[(c['enrolled'], c['instructor']) for c in course_by_size[:20]] 


[(784, ('Leah', 'Berkenwald', 'leahb@brandeis.edu')),
 (186, ('Stephanie', 'Murray', 'murray@brandeis.edu')),
 (186, ('Maria', 'de Boef Miara', 'mmiara@brandeis.edu')),
 (181, ('Maria', 'de Boef Miara', 'mmiara@brandeis.edu')),
 (180, ('Stephanie', 'Murray', 'murray@brandeis.edu')),
 (175, ('Stuart', 'Altman', 'altman@brandeis.edu')),
 (170, ('Anne S', 'Berry', 'anneberry@brandeis.edu')),
 (166, ('Ellen J', 'Wright', 'ejwright@brandeis.edu')),
 (166, ('Timothy J', 'Hickey', 'tjhickey@brandeis.edu')),
 (150, ('Timothy J', 'Hickey', 'tjhickey@brandeis.edu')),
 (145, ('Daniel', 'Breen', 'dbreen91@brandeis.edu')),
 (141, ('Dan L', 'Perlman', 'perlman@brandeis.edu')),
 (136, ('Colleen', 'Hitchcock', 'hitchcock@brandeis.edu')),
 (135, ('Teresa Vann', 'Mitchell', 'tmitch@brandeis.edu')),
 (119, ('Jennifer', 'Gutsell', 'jgutsell@brandeis.edu')),
 (109, ('Peter', 'Mistark', 'pmistark@brandeis.edu')),
 (107, ('Peter', 'Mistark', 'pmistark@brandeis.edu')),
 (105, ('Michael Thomas', 'Marr', 'mmarr

h)list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number) 

In [14]:
coures_by_student = sorted(courses, key = lambda course: -course['enrolled'])
[(c['enrolled'], c['subject'], c['coursenum']) for c in course_by_size[:20]] 

[(784, 'HWL', '1-PRE'),
 (186, 'CHEM', '25A'),
 (186, 'BIOL', '42A'),
 (181, 'BIOL', '15B'),
 (180, 'CHEM', '25B'),
 (175, 'HS', '104B'),
 (170, 'PSYC', '10A'),
 (166, 'PSYC', '10A'),
 (166, 'COSI', '164A'),
 (150, 'COSI', '10A'),
 (145, 'LGLS', '10A'),
 (141, 'ENVS', '2A'),
 (136, 'BIOL', '16A'),
 (135, 'NPSY', '199A'),
 (119, 'PSYC', '34B'),
 (109, 'PHYS', '10A'),
 (107, 'PHYS', '10B'),
 (105, 'BIOL', '350A'),
 (105, 'PSYC', '51A'),
 (104, 'ECON', '10A')]

i)

Ni Jian): list the top 3 instructors in terms of the number of students they taught in subject COSI

In [15]:
course_by_size = sorted((c for c in courses if c['subject'] =='COSI'), key = lambda course: -course['enrolled'])
[(c['enrolled'], c['instructor']) for c in course_by_size[:3]]                    

[(166, ('Timothy J', 'Hickey', 'tjhickey@brandeis.edu')),
 (150, ('Timothy J', 'Hickey', 'tjhickey@brandeis.edu')),
 (102, ('Antonella', 'DiLillo', 'dilant@brandeis.edu'))]

Lu Lu): List all the classes that meet on Mondays and Wednesdays.

In [14]:
# len([course for course in courses if 'mon' in course['times'][0]['days']])
for course in courses:
    if course['times']:
        if 'm'  in course['times'][0]['days'] and 'w' in course['times'][0]['days']:
            print(course['name'])

Digital Fabrication with Robotics
Python and Applications to Business Analytics
Analyzing Big Data I
Private Equity
Financial Statement Analysis
Analyzing Big Data I
Corporate Governance: From Colossal Failures to Best Practices
Chemistry Colloquium
Graduate Proseminar
Introduction to Doctoral Studies
Graduate Research Methods in Psychology
Approaches to Literary and Cultural Studies
Clinical Genetics II
Masters Proseminar
Chinese Pedagogy I
Seminar in Composition
Seminar in Composition
Statistics for Jewish Professional Leaders
Topology III: Vector Bundles and Applications
Algebraic Geometry I
Introduction to Algebraic Number Theory
Lie Algebras: Representation Theory
Foundations of Sociological Theory
Graduate Seminar in American Politics
Machine Learning and Data Analysis for Business and Finance
Business and Economic Strategies in Emerging Markets
Business Dynamics: Managing in a Complex World
Managerial Economics
Accounting and Financial Analysis
Accounting and Financial Analysis


Wantong Lin):

Vu Le):

Mahmoud Salah):